In [1]:
import requests
import pandas as pd
import time

In [2]:
q1 = '''
query (
  $page: Int = 1
  $sort: [MediaSort] = [SCORE_DESC]
  ) {
  Page(page: $page) {
    pageInfo {
      hasNextPage
    }
    media(
      type: ANIME
      sort: $sort
    ) {
      title {
        romaji 
        english
        
      }
      averageScore
      episodes
      status
      format
      genres
      season
      seasonYear
      popularity
      description
      studios{
        edges{
          isMain
          node{
            name
          }
        }
      }
      coverImage{
        large
        color
      }
      id
      tags{
        name
        rank
        category    
      }
      meanScore
      popularity
      favourites
      trending
      bannerImage
      streamingEpisodes{
        site
      }
      characters{
        edges{
          role
          node{
            name{
              full
            }
          }
        }
      }
      
    }
  }
}'''

# Define our query variables and values that will be used in the query request
variables = {
    'page': 1
}

url = 'https://graphql.anilist.co'
df = pd.DataFrame(columns=[
  'id','title','averageScore','episodes','status','format','genres','season','seasonYear', 'streamingPlatform', 'popularity', 
  'description', 'studio', 'coverImage', 'tags', 'meanScore', 'favourites', 'trending', 'bannerImage', 'mainCharacters'
])


In [3]:
while True:
  response = requests.post(url, json={'query':q1, 'variables': variables})
  while response.status_code != 200:
    time.sleep(5)
    response = requests.post(url, json={'query':q1, 'variables': variables})
    
  isNextPage = response.json()['data']['Page']['pageInfo']['hasNextPage']
  animes = response.json()['data']['Page']['media']
  for anime in animes:
    title = anime['title']['english'] or anime['title']['romaji']
    averageScore = str(anime['averageScore'])
    episodes = str(anime['episodes'])
    status = anime['status']
    format_ = anime['format']
    genres = ', '.join(anime['genres'])
    season = anime['season']
    seasonYear = str(anime['seasonYear'])
    popularity = str(anime['popularity'])
    description = anime['description']
    anime_studio = ""
    for stuio in anime['studios']['edges']:
      if stuio['isMain']:
        anime_studio = stuio['node']['name']
        break
    coverImage = anime['coverImage']['large']
    id = str(anime['id'])
    tags = anime['tags']
    meanScore = str(anime['meanScore'])
    favourites = str(anime['favourites'])
    trending = str(anime['trending'])
    bannerImage = anime['bannerImage']
    streamingPlatform = anime['streamingEpisodes'][0]['site'] if anime['streamingEpisodes'] else "N/A"
    mainCharcters = ",  ".join([ character['node']['name']['full'] for character in anime['characters']['edges'] if character['role'] == "MAIN"])
    df.loc[len(df)] = [
      id,title,averageScore,episodes,status,format_,genres,season,seasonYear,streamingPlatform,popularity,description, anime_studio,
      coverImage, tags, meanScore, favourites, trending, bannerImage, mainCharcters
    ]
    print(f"Added: {title}") 
    
  if not isNextPage:
    break
  
  variables['page'] += 1
  
    
  

Added: Frieren: Beyond Journey’s End
Added: Gintama: THE VERY FINAL
Added: Gintama Season 4
Added: ONE PIECE FAN LETTER
Added: Fullmetal Alchemist: Brotherhood
Added: Fruits Basket The Final Season
Added: Attack on Titan Season 3 Part 2
Added: Owarimonogatari Second Season
Added: Hunter x Hunter (2011)
Added: Steins;Gate
Added: Kaguya-sama: Love is War -Ultra Romantic-
Added: March comes in like a lion Season 2
Added: Gintama Season 3
Added: Gintama Season 2
Added: Gintama Season 5
Added: Tomorrow's Joe 2
Added: BLEACH: Thousand-Year Blood War
Added: Legend of the Galactic Heroes
Added: The Apothecary Diaries Season 2
Added: The Apothecary Diaries
Added: Vinland Saga Season 2
Added: Monster
Added: A Silent Voice
Added: Monogatari Series Second Season
Added: ONE PIECE
Added: Mob Psycho 100 II
Added: Takopi's Original Sin
Added: Gintama.: Silver Soul Arc - Second Half War
Added: Gintama: The Final Chapter - Be Forever Yorozuya
Added: Sound! Euphonium 3
Added: Vinland Saga
Added: Clannad:

TypeError: sequence item 8: expected str instance, NoneType found

In [4]:
variables['page']
print(response)

<Response [200]>


In [5]:
df.head()

,id,title,averageScore,episodes,status,format,genres,season,seasonYear,streamingPlatform,popularity,description,studio,coverImage,tags,meanScore,favourites,trending,bannerImage,mainCharacters
0,154587,Frieren: Beyond Journey’s End,91,28,FINISHED,TV,"Adventure, Drama, Fantasy",FALL,2023,Crunchyroll,333256,The adventure is over but life goes on for an ...,MADHOUSE,https://s4.anilist.co/file/anilistcdn/media/an...,"[{'name': 'Travel', 'rank': 96, 'category': 'T...",91,40623,21,https://s4.anilist.co/file/anilistcdn/media/an...,"Frieren, Stark, Fern"
1,114129,Gintama: THE VERY FINAL,91,1,FINISHED,MOVIE,"Action, Comedy, Drama, Sci-Fi",WINTER,2021,Crunchyroll,47089,<i>Gintama: THE FINAL</i> is the 3rd and final...,Bandai Namco Pictures,https://s4.anilist.co/file/anilistcdn/media/an...,"[{'name': 'Samurai', 'rank': 94, 'category': '...",91,2949,0,https://s4.anilist.co/file/anilistcdn/media/an...,"Gintoki Sakata, Kagura, Shinpachi Shimura, ..."
2,20996,Gintama Season 4,90,51,FINISHED,TV,"Action, Comedy, Drama, Sci-Fi",SPRING,2015,Crunchyroll,107645,"Gintoki, Shinpachi, and Kagura return as the f...",Bandai Namco Pictures,https://s4.anilist.co/file/anilistcdn/media/an...,"[{'name': 'Samurai', 'rank': 89, 'category': '...",91,5805,0,https://s4.anilist.co/file/anilistcdn/media/an...,"Gintoki Sakata, Kagura, Shinpachi Shimura"
3,182469,ONE PIECE FAN LETTER,90,1,FINISHED,SPECIAL,"Action, Adventure, Fantasy",FALL,2024,N/A,38918,To commemorate the 25th anniversary of the <i>...,Toei Animation,https://s4.anilist.co/file/anilistcdn/media/an...,"[{'name': 'Female Protagonist', 'rank': 91, 'c...",90,2390,0,https://s4.anilist.co/file/anilistcdn/media/an...,
4,5114,Fullmetal Alchemist: Brotherhood,90,64,FINISHED,TV,"Action, Adventure, Drama, Fantasy",SPRING,2009,Crunchyroll,625860,"""In order for something to be obtained, someth...",bones,https://s4.anilist.co/file/anilistcdn/media/an...,"[{'name': 'Alchemy', 'rank': 96, 'category': '...",90,54289,16,https://s4.anilist.co/file/anilistcdn/media/an...,"Alphonse Elric, Edward Elric"


In [6]:
dataset_location = '../Datasets/anilist.csv'
df.to_csv(dataset_location, index=False)